In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|default_exp envs.gym

In [ ]:
#|export
# Python native modules
import os
import warnings
from typing import Callable, Any, Union, Iterable, Optional
# Third party libs
import gymnasium as gym
import torch
# from fastrl.torch_core import *
from fastcore.all import add_docs
import torchdata.datapipes as dp
from torchdata.dataloader2.graph import find_dps,DataPipeGraph,DataPipe,traverse_dps
from torchdata.dataloader2 import MultiProcessingReadingService
from torchdata.datapipes.iter import IterDataPipe
from torchdata.datapipes.map import MapDataPipe
# Local modules
from fastrl.core import StepType,SimpleStep
from fastrl.pipes.core import find_dps
from fastrl.pipes.iter.nskip import NSkipper
from fastrl.pipes.iter.nstep import NStepper,NStepFlattener
from fastrl.pipes.iter.firstlast import FirstLastMerger
# from fastrl.pipes.iter.transforms import *
# from fastrl.pipes.map.transforms import *
# from fastrl.data.block import DataPipeAugmentationFn

# Envs Gym
> Fastrl API for working with OpenAI Gyms

### Pipes

In [ ]:
#|export
class GymStepper(dp.iter.IterDataPipe):
    def __init__(self,
        source_datapipe:Union[Iterable,dp.iter.IterDataPipe], # Calling `next()` should produce a `gym.Env`
        agent=None, # Optional `Agent` that accepts a `SimpleStep` to produce a list of actions.
        seed:int=None, # Optional seed to set the env to and also random action sames if `agent==None`
        synchronized_reset:bool=False, # Some `gym.Envs` require reset to be terminated on *all* envs before proceeding to step.
        include_images:bool=False, # Render images from the environment
        terminate_on_truncation:bool=True
    ):
        self.source_datapipe = source_datapipe
        self.agent = agent
        self.seed = seed
        self.include_images = include_images
        self.synchronized_reset = synchronized_reset
        self.terminate_on_truncation = terminate_on_truncation
        self._env_ids = {}
        
    def env_reset(self,
      env:gym.Env, # The env to rest along with its numeric object id
      env_id:int # Resets env in `self._env_ids[env_id]`
    ) -> StepType:
        state, info = env.reset(seed=self.seed)
        env.action_space.seed(seed=self.seed)
        episode_n = self._env_ids[env_id].episode_n+1 if env_id in self._env_ids else torch.tensor(1)

        step = (self.no_agent_create_step if self.agent is None else self.agent.create_step)(
            state=torch.tensor(state),
            next_state=torch.tensor(state),
            terminated=torch.tensor(False),
            truncated=torch.tensor(False),
            reward=torch.tensor(0),
            total_reward=torch.tensor(0.),
            env_id=torch.tensor(env_id),
            proc_id=torch.tensor(os.getpid()),
            step_n=torch.tensor(0),
            episode_n=episode_n,
            # image=env.render(mode='rgb_array') if self.include_images else torch.FloatTensor([0])
            image=env.render() if self.include_images else torch.FloatTensor([0])
        )
        self._env_ids[env_id] = step
        return step
    
    def no_agent_create_step(self,**kwargs): return SimpleStep(**kwargs)

    def __iter__(self) -> SimpleStep:
        for env in self.source_datapipe:
            assert issubclass(env.__class__,gym.Env),f'Expected subclass of gym.Env, but got {env.__class__}'    
            env_id = id(env)
            
            if env_id not in self._env_ids or self._env_ids[env_id].terminated:
                if self.synchronized_reset:
                    if env_id in self._env_ids \
                       and not self._env_ids[env_id].terminated \
                       and self._resetting_all:
                        # If this env has already been reset, and we are currently in the 
                        # self._resetting_all phase, then skip this so we can reset all remaining envs
                        continue
                    elif env_id not in self._env_ids \
                       or all([self._env_ids[s].terminated for s in self._env_ids])\
                       or self._resetting_all:
                        # If the id is not in the _env_ids, we can assume this is a fresh start.
                        # OR 
                        # If all the envs are terminated, then we can start doing a reset operation.
                        # OR
                        # If we are currently resetting all the envs anyways
                        # This means we want to reset ALL the envs before doing any steps.
                        self.env_reset(env,env_id)
                        # Move to the next env, eventually we will reset all the envs in sync.
                        # then we will be able to start calling `step` for each of them.
                        # _resetting_all is True when there are envs still "terminated".
                        self._resetting_all = any([self._env_ids[s].terminated for s in self._env_ids])
                        continue 
                    elif self._env_ids[env_id].terminated:
                        continue
                    else:
                        raise ValueError('This else should never happen.')
                else:
                    step = self.env_reset(env,env_id)
            else:
                step = self._env_ids[env_id]

            action = None
            for action in (self.agent([step]) if self.agent is not None else [env.action_space.sample()]):
                next_state,reward,terminated,truncated,_ = env.step(
                    self.agent.augment_actions(action) if self.agent is not None else action
                )

                if self.terminate_on_truncation and truncated: terminated = True

                step = (self.no_agent_create_step if self.agent is None else self.agent.create_step)(
                    state=torch.tensor(step.next_state),
                    next_state=torch.tensor(next_state),
                    action=torch.tensor(action).float(),
                    terminated=torch.tensor(terminated),
                    truncated=torch.tensor(truncated),
                    reward=torch.tensor(reward),
                    total_reward=step.total_reward+reward,
                    env_id=torch.tensor(env_id),
                    proc_id=torch.tensor(os.getpid()),
                    step_n=step.step_n+1,
                    episode_n=step.episode_n,
                    # image=env.render(mode='rgb_array') if self.include_images else torch.FloatTensor([0])
                    image=env.render() if self.include_images else torch.FloatTensor([0])
                )
                self._env_ids[env_id] = step
                yield step
                if terminated: break
            if action is None: 
                raise Exception('The agent produced no actions. This should never occur.')
                
add_docs(
    GymStepper,
    """Accepts a `source_datapipe` or iterable whose `next()` produces a single `gym.Env`.
       Tracks multiple envs using `id(env)`.""",
    env_reset="Resets a env given the env_id.",
    no_agent_create_step="If there is no agent for creating the step output, then `GymStepper` will create its own"
)

In [ ]:
#|hide
# Used here to avoid UserWarnings related to gym complaining about bounding box / action space format.
# There must be a bug in the CartPole-v1 env that is causing this to show. Also couldnt figure out the 
# regex, so instead we filter on the lineno, which is line 98.
warnings.filterwarnings("ignore",category=UserWarning,lineno=98)

## Iteration Examples

In [ ]:
import pandas as pd
from fastrl.agents.core import * 

In [ ]:
#|eval: False
class ConstantRunner(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,constant=1,array_nestings=0): 
        self.source_datapipe = source_datapipe
        self.agent_base = find_dps(traverse_dps(self.source_datapipe),AgentBase)
        self.constant = constant
        self.array_nestings = array_nestings
    
    def __iter__(self):
        for o in self.source_datapipe: 
            try: 
                if self.array_nestings==0: yield self.constant
                else:
                    yield [self.constant]*self.array_nestings
            except Exception:
                print('Failed on ',o)
                raise

agent = AgentBase(None,[])
agent = ConstantRunner(agent)
agent = AgentHead(agent)

pipe = dp.map.Mapper(['CartPole-v1']*3)
pipe = pipe.map(lambda o:gym.make(o,render_mode='rgb_array'))
# pipe = TypeTransformer(pipe,[GymTypeTransform])
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle()
pipe = GymStepper(pipe,agent=agent,seed=0)

pd.DataFrame([step for step,_ in zip(*(pipe,range(10)))])[['state','next_state','action','terminated']]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/datapipes/utils/common.py:149: UserWarning: Lambda function is not supported by pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


,state,next_state,action,terminated
0,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False)
1,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False)
2,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False)
3,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False)
4,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False)
5,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False)
6,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False)
7,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False)
8,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False)
9,"[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]","[tensor(0.0353), tensor(0.7603), tensor(-0.0866), tensor(-1.2844)]",tensor(1.),tensor(False)


In [ ]:
from functools import partial

In [ ]:
pipe = dp.map.Mapper(['CartPole-v1']*3)
pipe = pipe.map(partial(gym.make,render_mode='rgb_array'))
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle()
pipe = GymStepper(pipe,seed=0)

pd.DataFrame([step for step,_ in zip(*(pipe,range(10)))])[['state','next_state','action','terminated','env_id']]

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


,state,next_state,action,terminated,env_id
0,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False),tensor(140328573563520)
1,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False),tensor(140328576599328)
2,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False),tensor(140328573879440)
3,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False),tensor(140328573563520)
4,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False),tensor(140328576599328)
5,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False),tensor(140328573879440)
6,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False),tensor(140328573563520)
7,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False),tensor(140328576599328)
8,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False),tensor(140328573879440)
9,"[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]","[tensor(0.0353), tensor(0.3702), tensor(-0.0866), tensor(-0.7006)]",tensor(0.),tensor(False),tensor(140328573563520)


In [ ]:
from torchdata.dataloader2 import DataLoader2,MultiProcessingReadingService

In [ ]:
def seed_worker(pipe,worker_info): 
    torch.manual_seed(0)
    return pipe

dl = DataLoader2(pipe,reading_service=MultiProcessingReadingService(
        num_workers = 1,
        worker_init_fn=seed_worker
    )
)

pd.DataFrame([step for step,_ in zip(*(dl,range(10)))])[['state','next_state','action','terminated','env_id']]

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


,state,next_state,action,terminated,env_id
0,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False),tensor(140328573727888)
1,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False),tensor(140328572973600)
2,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]",tensor(1.),tensor(False),tensor(140328572973744)
3,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False),tensor(140328573727888)
4,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False),tensor(140328572973600)
5,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(1.),tensor(False),tensor(140328572973744)
6,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False),tensor(140328573727888)
7,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False),tensor(140328572973600)
8,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(1.),tensor(False),tensor(140328572973744)
9,"[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]","[tensor(0.0353), tensor(0.3702), tensor(-0.0866), tensor(-0.7006)]",tensor(0.),tensor(False),tensor(140328573727888)


In [ ]:
pipe = dp.map.Mapper(['CartPole-v1']*3)
pipe = pipe.map(partial(gym.make,render_mode='rgb_array'))
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle()
pipe = GymStepper(pipe,synchronized_reset=True)

pd.DataFrame([step for step,_ in zip(*(dl,range(10)))])[['state','next_state','action','terminated']]

,state,next_state,action,terminated
0,"[tensor(0.0137), tensor(-0.2046), tensor(-0.0875), tensor(-0.0543)]","[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]",tensor(1.),tensor(False)
1,"[tensor(0.0137), tensor(-0.2046), tensor(-0.0875), tensor(-0.0543)]","[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]",tensor(1.),tensor(False)
2,"[tensor(0.0217), tensor(-0.4009), tensor(-0.0929), tensor(0.2661)]","[tensor(0.0137), tensor(-0.2046), tensor(-0.0875), tensor(-0.0543)]",tensor(1.),tensor(False)
3,"[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]","[tensor(0.0094), tensor(0.1879), tensor(-0.0961), tensor(-0.6926)]",tensor(1.),tensor(False)
4,"[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]","[tensor(0.0094), tensor(0.1879), tensor(-0.0961), tensor(-0.6926)]",tensor(1.),tensor(False)
5,"[tensor(0.0137), tensor(-0.2046), tensor(-0.0875), tensor(-0.0543)]","[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]",tensor(1.),tensor(False)
6,"[tensor(0.0094), tensor(0.1879), tensor(-0.0961), tensor(-0.6926)]","[tensor(0.0132), tensor(0.3842), tensor(-0.1099), tensor(-1.0139)]",tensor(1.),tensor(False)
7,"[tensor(0.0094), tensor(0.1879), tensor(-0.0961), tensor(-0.6926)]","[tensor(0.0132), tensor(0.3842), tensor(-0.1099), tensor(-1.0139)]",tensor(1.),tensor(False)
8,"[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]","[tensor(0.0094), tensor(0.1879), tensor(-0.0961), tensor(-0.6926)]",tensor(1.),tensor(False)
9,"[tensor(0.0132), tensor(0.3842), tensor(-0.1099), tensor(-1.0139)]","[tensor(0.0209), tensor(0.5806), tensor(-0.1302), tensor(-1.3390)]",tensor(1.),tensor(False)


## Tests

We create 3 envs and put a max iteration count at 180. Each env will run for 18 steps before ending, which means
we expect there to be 10 total episodes.

In [ ]:
envs = ['CartPole-v1']*3
n_episodes = 3

pipe = dp.map.Mapper(envs)
pipe = pipe.map(partial(gym.make,render_mode='rgb_array'))
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
# We want to cycle through the envs enough times that their epsiode sum to 9, 3 episodes each
pipe = pipe.cycle(count=(18*len(envs))) 
pipe = GymStepper(pipe,seed=0)

All the of the environments should reach max 18 steps given a seed of 0...\
The total number of iterations should be `( 18 * n_envs) * n_episodes_per_env = 162`...

In [ ]:
from fastrl.core import test_len
from fastcore.all import test_eq,test_ne
from itertools import groupby

In [ ]:
steps = list(pipe)
gsteps = dict(groupby(steps,lambda o:int(o.step_n)))
test_len(gsteps.keys(),18)
pd.DataFrame([step for step in steps])[['state','terminated','env_id','episode_n','step_n']][::10]

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


,state,terminated,env_id,episode_n,step_n
0,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]",tensor(False),tensor(140328573566832),tensor(1),tensor(1)
10,"[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(False),tensor(140328573067120),tensor(1),tensor(4)
20,"[tensor(0.0463), tensor(-0.0172), tensor(-0.1094), tensor(-0.1771)]",tensor(False),tensor(140328572961312),tensor(1),tensor(7)
30,"[tensor(0.0217), tensor(-0.4009), tensor(-0.0929), tensor(0.2661)]",tensor(False),tensor(140328573566832),tensor(1),tensor(11)
40,"[tensor(0.0094), tensor(0.1879), tensor(-0.0961), tensor(-0.6926)]",tensor(False),tensor(140328573067120),tensor(1),tensor(14)
50,"[tensor(0.0325), tensor(0.7771), tensor(-0.1570), tensor(-1.6694)]",tensor(False),tensor(140328572961312),tensor(1),tensor(17)
60,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(False),tensor(140328573566832),tensor(2),tensor(3)
70,"[tensor(0.0427), tensor(0.1763), tensor(-0.1007), tensor(-0.4364)]",tensor(False),tensor(140328573067120),tensor(2),tensor(6)
80,"[tensor(0.0417), tensor(-0.4040), tensor(-0.1113), tensor(0.3342)]",tensor(False),tensor(140328572961312),tensor(2),tensor(9)
90,"[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]",tensor(False),tensor(140328573566832),tensor(2),tensor(13)


All of the step groups should be the same length...

In [ ]:
group_sz = None
gsteps = {k:list(v) for k,v in groupby(steps,lambda o:int(o.step_n))}
for name,group in gsteps.items():
    if group_sz is None: group_sz = len(group)
    else:                assert len(group)==group_sz,f' Got lengths {len(group)} and {group_sz} for {name}.\n\n{group}'

Each step group's state and next_states should match across envs...

In [ ]:
group_sz = None
for name,group in gsteps.items():
    e1 = group[0]
    for other in group[1:]: test_eq(e1.state,other.state)
    for other in group[1:]: test_eq(e1.next_state,other.next_state)

Each step group value should not show up/be duplicated in any other step groups...

In [ ]:
group_sz = None
for name,group in gsteps.items():
    e1 = group[0]
    for other_name,other_group in gsteps.items():
        if other_name==name: continue
        for other in other_group[1:]: test_ne(e1.state,other.state)
        for other in other_group[1:]: test_ne(e1.next_state,other.next_state)

Given 3 envs, single steps, epsiodes of 18 steps in len, 3 episodes each, run for 162 iterations, we should
expect there to be 9 dones.

In [ ]:
import torch
from fastcore.all import L,Self

In [ ]:
test_eq(sum([o.terminated for o in steps]),torch.tensor([9]))

The max episode numbers for each env should sum to 9 where for each env, it should reach and finish 3 episodes...

In [ ]:
gsteps = {k:list(v) for k,v in groupby(steps,lambda o:int(o.env_id))}
test_len(gsteps.keys(),3)
env1,env2,env3 = L(gsteps.values()).map(L).map(Self.map(Self.episode_n()).map(int))
test_eq(max(env1)+max(env2)+max(env3),9)

### Test the `synchronized_reset` param...
> In this case, we will have iterate through the 3 envs without producing a step on warmup.

In [ ]:
envs = ['CartPole-v1']*3
n_episodes = 3

pipe = dp.map.Mapper(envs)
pipe = pipe.map(partial(gym.make,render_mode='rgb_array'))
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
# We want to cycle through the envs enough times that their epsiode sum to 9, 3 episodes each
# We add an additional +3 cycles since `synchronized_reset` cycles through the envs additional times
# to make sure they are all reset prior to stepping
pipe = pipe.cycle(count=(18*len(envs))+3) 
pipe = GymStepper(pipe,seed=0,synchronized_reset=True)

In [ ]:
steps = list(pipe)
gsteps = {k:list(v) for k,v in groupby(steps,lambda o:int(o.step_n))}
test_len(gsteps.keys(),18)
pd.DataFrame([step for step in steps])[['state','terminated','env_id','episode_n','step_n']][::10]

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


,state,terminated,env_id,episode_n,step_n
0,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]",tensor(False),tensor(140328572518752),tensor(1),tensor(1)
10,"[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(False),tensor(140328573336544),tensor(1),tensor(4)
20,"[tensor(0.0463), tensor(-0.0172), tensor(-0.1094), tensor(-0.1771)]",tensor(False),tensor(140328572520528),tensor(1),tensor(7)
30,"[tensor(0.0217), tensor(-0.4009), tensor(-0.0929), tensor(0.2661)]",tensor(False),tensor(140328572518752),tensor(1),tensor(11)
40,"[tensor(0.0094), tensor(0.1879), tensor(-0.0961), tensor(-0.6926)]",tensor(False),tensor(140328573336544),tensor(1),tensor(14)
50,"[tensor(0.0325), tensor(0.7771), tensor(-0.1570), tensor(-1.6694)]",tensor(False),tensor(140328572520528),tensor(1),tensor(17)
60,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(False),tensor(140328572518752),tensor(2),tensor(3)
70,"[tensor(0.0427), tensor(0.1763), tensor(-0.1007), tensor(-0.4364)]",tensor(False),tensor(140328573336544),tensor(2),tensor(6)
80,"[tensor(0.0417), tensor(-0.4040), tensor(-0.1113), tensor(0.3342)]",tensor(False),tensor(140328572520528),tensor(2),tensor(9)
90,"[tensor(0.0096), tensor(-0.0083), tensor(-0.0886), tensor(-0.3733)]",tensor(False),tensor(140328572518752),tensor(2),tensor(13)


All of the step groups should be the same length...

In [ ]:
group_sz = None
for name,group in gsteps.items():
    if group_sz is None: group_sz = len(group)
    else:                assert len(group)==group_sz,f' Got lengths {len(group)} and {group_sz} for {name}.\n\n{group}'

Each step group's state and next_states should match across envs...

In [ ]:
group_sz = None
for name,group in gsteps.items():
    e1 = group[0]
    for other in group[1:]: test_eq(e1.state,other.state)
    for other in group[1:]: test_eq(e1.next_state,other.next_state)

Each step group value should not show up/be duplicated in any other step groups...

In [ ]:
group_sz = None
for name,group in gsteps.items():
    e1 = group[0]
    for other_name,other_group in gsteps.items():
        if other_name==name: continue
        for other in other_group[1:]: test_ne(e1.state,other.state)
        for other in other_group[1:]: test_ne(e1.next_state,other.next_state)

Given 3 envs, single steps, epsiodes of 18 steps in len, 3 episodes each, run for 162 iterations, we should
expect there to be 9 dones.

In [ ]:
test_eq(sum([o.terminated for o in steps]),torch.tensor([9]))

The max episode numbers for each env should sum to 9 where for each env, it should reach and finish 3 episodes...

In [ ]:
# gsteps = groupby(steps,lambda o:int(o.env_id))
gsteps = {k:list(v) for k,v in groupby(steps,lambda o:int(o.env_id))}
test_len(gsteps.keys(),3)
env1,env2,env3 = L(gsteps.values()).map(L).map(Self.map(Self.episode_n()).map(int))
test_eq(max(env1)+max(env2)+max(env3),9)

In [ ]:
envs = ['CartPole-v1']*10

pipe = dp.map.Mapper(envs)
pipe = pipe.map(partial(gym.make,render_mode='rgb_array'))
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle(count=(18*len(envs))) 
# Turn off the seed so that some envs end before others...
pipe = GymStepper(pipe,synchronized_reset=True)
steps = list(pipe)

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


Since the seed is turned off the only properties we are to expect are:
    
    - If an env finishes, no steps from that env should be seen until all 9 of the other envs finish

In [ ]:
def synchronized_reset_checker(steps):
    env_id_done_tracker = {}
    did_syncs_happen = False
    for d,env_id,idx in [(bool(o.terminated),int(o.env_id),i) for i,o in enumerate(steps)]:

        if d: 
            env_id_done_tracker[env_id] = idx
            continue

        if env_id in env_id_done_tracker:
            if len(env_id_done_tracker)!=len(envs):
                raise Exception(f'env_id {env_id} was iterated through when it should not have been! idx: {idx}')
        if len(env_id_done_tracker)==len(envs):
            did_syncs_happen = True
            env_id_done_tracker = {}

    if not did_syncs_happen: 
        raise Exception('There should have at least been 1 time where all the envs had to reset, which did not happen.')
synchronized_reset_checker(steps)

For sanity, we should expect that without `synchronized_reset` envs will be reset and stepped through before other 
envs are reset, `synchronized_reset_checker` should fail.

In [ ]:
pipe = dp.map.Mapper(envs)
pipe = pipe.map(partial(gym.make,render_mode='rgb_array'))
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle(count=(18*len(envs))) 
# Turn off the seed so that some envs end before others...
pipe = GymStepper(pipe)
steps = list(pipe)

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


In [ ]:
from fastcore.test import ExceptionExpected

In [ ]:
with ExceptionExpected(regex='was iterated through when it should not have been'):
    synchronized_reset_checker(steps)

In [ ]:
# #|export
# class GymTransformBlock():

#     def __init__(self,
#         agent:DataPipe, # An AgentHead
#         seed:Optional[int]=None, # The seed for the gym to use
#         # Used by `NStepper`, outputs tuples / chunks of assiciated steps
#         nsteps:int=1, 
#         # Used by `NSkipper` to skip a certain number of steps (agent still gets called for each)
#         nskips:int=1,
#         # Whether when nsteps>1 to merge it into a single `StepType`
#         firstlast:bool=False,
#         # Functions to run once, at the beginning of the pipeline
#         type_tfms:Optional[List[Callable]]=None,
#         # Functions to run over individual steps before batching
#         item_tfms:Optional[List[Callable]]=None,
#         # Functions to run over batches (as specified by `bs`)
#         batch_tfms:Optional[List[Callable]]=None,
#         # The batch size, which is different from `nsteps` in that firstlast will be 
#         # run prior to batching, and a batch of steps might come from multiple envs,
#         # where nstep is associated with a single env
#         bs:int=1,
#         # The prefered default is for the pipeline to be infinate, and the learner
#         # decides how much to iter. If this is not None, then the pipeline will run for 
#         # that number of `n`
#         n:Optional[int]=None,
#         # Whether to reset all the envs at the same time as opposed to reseting them 
#         # the moment an episode ends. 
#         synchronized_reset:bool=False,
#         # Should be used only for validation / logging, will grab a render of the gym
#         # and assign to the `StepType` image field. This data should not be used for training.
#         # If it images are needed for training, then you should wrap the env instead. 
#         include_images:bool=False,
#         # If an environment truncates, terminate it.
#         terminate_on_truncation:bool=True,
#         # Additional pipelines to insert, replace, remove
#         dp_augmentation_fns:Tuple[DataPipeAugmentationFn]=None
#     ) -> None:
#         "Basic OpenAi gym `DataPipeGraph` with first-last, nstep, and nskip capability"
#         self.agent = agent
#         store_attr()

#     def __call__(
#         self,
#         # `source` likely will be an iterable that gets pushed into the pipeline when an 
#         # experiment is actually being run.
#         source:Any,
#         # Any parameters needed for the dataloader
#         num_workers:int=0,
#         # This param must exist: as_dataloader for the datablock to create dataloaders
#         as_dataloader:bool=False
#     ) -> DataPipeOrDataLoader:
#         _type_tfms = ifnone(self.type_tfms,GymTypeTransform)
#         "This is the function that is actually run by `DataBlock`"
#         pipe = dp.map.Mapper(source)
#         pipe = TypeTransformer(pipe,_type_tfms)
#         pipe = dp.iter.MapToIterConverter(pipe)
#         pipe = dp.iter.InMemoryCacheHolder(pipe)
#         pipe = pipe.cycle() # Cycle through the envs inf
#         pipe = GymStepper(pipe,agent=self.agent,seed=self.seed,
#                           include_images=self.include_images,
#                           terminate_on_truncation=self.terminate_on_truncation,
#                           synchronized_reset=self.synchronized_reset)
#         if self.nskips!=1: pipe = NSkipper(pipe,n=self.nskips)
#         if self.nsteps!=1:
#             pipe = NStepper(pipe,n=self.nsteps)
#             if self.firstlast:
#                 pipe = FirstLastMerger(pipe)
#             else:
#                 pipe = NStepFlattener(pipe) # We dont want to flatten if using FirstLastMerger
#         if self.n is not None: pipe = pipe.header(limit=self.n)
#         pipe = ItemTransformer(pipe,self.item_tfms)
#         pipe  = pipe.batch(batch_size=self.bs)
#         pipe = BatchTransformer(pipe,self.batch_tfms)
        
#         pipe = apply_dp_augmentation_fns(pipe,ifnone(self.dp_augmentation_fns,()))
        
#         if as_dataloader:
#             pipe = DataLoader2(
#                 datapipe=pipe,
#                 reading_service=PrototypeMultiProcessingReadingService(
#                     num_workers = num_workers,
#                     protocol_client_type = InputItemIterDataPipeQueueProtocolClient,
#                     protocol_server_type = InputItemIterDataPipeQueueProtocolServer,
#                     pipe_type = item_input_pipe_type,
#                     eventloop = SpawnProcessForDataPipeline
#                 ) if num_workers>0 else None
#             )
#         return pipe

#|export
def GymDataPipe(
    agent:DataPipe, # An AgentHead
    seed:Optional[int]=None, # The seed for the gym to use
    # Used by `NStepper`, outputs tuples / chunks of assiciated steps
    nsteps:int=1, 
    # Used by `NSkipper` to skip a certain number of steps (agent still gets called for each)
    nskips:int=1,
    # Whether when nsteps>1 to merge it into a single `StepType`
    firstlast:bool=False,
    # The batch size, which is different from `nsteps` in that firstlast will be 
    # run prior to batching, and a batch of steps might come from multiple envs,
    # where nstep is associated with a single env
    bs:int=1,
    # The prefered default is for the pipeline to be infinate, and the learner
    # decides how much to iter. If this is not None, then the pipeline will run for 
    # that number of `n`
    n:Optional[int]=None,
    # Whether to reset all the envs at the same time as opposed to reseting them 
    # the moment an episode ends. 
    synchronized_reset:bool=False,
    # Should be used only for validation / logging, will grab a render of the gym
    # and assign to the `StepType` image field. This data should not be used for training.
    # If it images are needed for training, then you should wrap the env instead. 
    include_images:bool=False,
    # If an environment truncates, terminate it.
    terminate_on_truncation:bool=True
) -> Callable:
    "Basic `gymnasium` `DataPipeGraph` with first-last, nstep, and nskip capability"

    def pipe_init(source,as_dataloader=False,num_workers=0):
        "This is the function that is actually run by `DataBlock`"
        pipe = dp.map.Mapper(source)
        pipe = pipe.map(gym.make)
        pipe = dp.iter.MapToIterConverter(pipe)
        pipe = dp.iter.InMemoryCacheHolder(pipe)
        pipe = pipe.cycle() # Cycle through the envs inf
        pipe = GymStepper(pipe,agent=agent,seed=seed,
                            include_images=include_images,
                            terminate_on_truncation=terminate_on_truncation,
                            synchronized_reset=synchronized_reset)
        if nskips!=1: pipe = NSkipper(pipe,n=nskips)
        if nsteps!=1:
            pipe = NStepper(pipe,n=nsteps)
            if firstlast:
                pipe = FirstLastMerger(pipe)
            else:
                pipe = NStepFlattener(pipe) # We dont want to flatten if using FirstLastMerger
        if n is not None: pipe = pipe.header(limit=n)
        pipe  = pipe.batch(batch_size=bs)
        
        if as_dataloader:
            pipe = DataLoader2(
                datapipe=pipe,
                reading_service=MultiProcessingReadingService(
                    num_workers = num_workers
                ) if num_workers > 0 else None
            )
        return pipe
    return pipe_init

In [ ]:
pd.set_option('display.max_rows', 50)

envs = ['CartPole-v1']*3
block = GymDataPipe(None,nsteps=2,nskips=2,firstlast=True,bs=1,n=100)
pipes = block(envs)
pd.DataFrame([o[0] for o in pipes])[['state','action','terminated','reward']][:50]

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),
/home/fastrl_user/fastrl/fastrl/pipes/iter/firstlast.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(fstep.state),
/home/fastrl_user/fastrl/fastrl/pipes/iter/firstlast.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state=torch.tensor(lstep.next_state),


,state,action,terminated,reward
0,"[tensor(0.0375), tensor(-0.0282), tensor(0.0318), tensor(-0.0246)]",tensor(1.),tensor(False),tensor(1.9900)
1,"[tensor(0.0152), tensor(0.0142), tensor(0.0102), tensor(-0.0116)]",tensor(0.),tensor(False),tensor(1.9900)
2,"[tensor(0.0029), tensor(-0.0042), tensor(0.0487), tensor(0.0330)]",tensor(1.),tensor(False),tensor(1.9900)
3,"[tensor(0.0370), tensor(0.1664), tensor(0.0313), tensor(-0.3071)]",tensor(1.),tensor(False),tensor(1.9900)
4,"[tensor(0.0155), tensor(-0.1810), tensor(0.0100), tensor(0.2843)]",tensor(1.),tensor(False),tensor(1.9900)
5,"[tensor(0.0028), tensor(0.1902), tensor(0.0493), tensor(-0.2439)]",tensor(0.),tensor(False),tensor(1.9900)
6,"[tensor(0.0475), tensor(0.5559), tensor(0.0134), tensor(-0.8744)]",tensor(0.),tensor(False),tensor(1.9900)
7,"[tensor(0.0121), tensor(0.2088), tensor(0.0156), tensor(-0.2929)]",tensor(0.),tensor(False),tensor(1.9900)
8,"[tensor(0.0065), tensor(0.1889), tensor(0.0457), tensor(-0.2145)]",tensor(1.),tensor(False),tensor(1.9900)
9,"[tensor(0.0658), tensor(0.1655), tensor(-0.0156), tensor(-0.2861)]",tensor(0.),tensor(False),tensor(1.9900)


In [ ]:
pd.set_option('display.max_rows', 50)

envs = ['CartPole-v1']*3
block = GymDataPipe(None,nsteps=1,nskips=1,firstlast=True,bs=1,n=100)
pipes = block(envs)

pd.DataFrame([o[0] for o in pipes])[['state','action','terminated','reward']][:50]

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


,state,action,terminated,reward
0,"[tensor(-0.0414), tensor(0.0239), tensor(-0.0089), tensor(0.0348)]",tensor(1.),tensor(False),tensor(1.)
1,"[tensor(-0.0117), tensor(0.0489), tensor(0.0266), tensor(0.0155)]",tensor(0.),tensor(False),tensor(1.)
2,"[tensor(-0.0090), tensor(0.0255), tensor(-0.0366), tensor(-0.0043)]",tensor(0.),tensor(False),tensor(1.)
3,"[tensor(-0.0409), tensor(0.2191), tensor(-0.0082), tensor(-0.2606)]",tensor(1.),tensor(False),tensor(1.)
4,"[tensor(-0.0107), tensor(-0.1466), tensor(0.0269), tensor(0.3164)]",tensor(0.),tensor(False),tensor(1.)
5,"[tensor(-0.0085), tensor(-0.1691), tensor(-0.0366), tensor(0.2767)]",tensor(0.),tensor(False),tensor(1.)
6,"[tensor(-0.0365), tensor(0.4144), tensor(-0.0134), tensor(-0.5559)]",tensor(1.),tensor(False),tensor(1.)
7,"[tensor(-0.0136), tensor(-0.3421), tensor(0.0332), tensor(0.6175)]",tensor(0.),tensor(False),tensor(1.)
8,"[tensor(-0.0118), tensor(-0.3637), tensor(-0.0311), tensor(0.5576)]",tensor(1.),tensor(False),tensor(1.)
9,"[tensor(-0.0282), tensor(0.6097), tensor(-0.0245), tensor(-0.8527)]",tensor(1.),tensor(False),tensor(1.)


In [ ]:
envs = ['CartPole-v1']*3
block = GymDataPipe(None,nsteps=2,nskips=1,firstlast=True,bs=1,n=100)
pipes = block(envs)
pd.DataFrame([o[0] for o in pipes])[['state','action','terminated','reward']][:50]

/tmp/ipykernel_22366/3616410577.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),
/home/fastrl_user/fastrl/fastrl/pipes/iter/firstlast.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(fstep.state),
/home/fastrl_user/fastrl/fastrl/pipes/iter/firstlast.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state=torch.tensor(lstep.next_state),


,state,action,terminated,reward
0,"[tensor(0.0468), tensor(-0.0144), tensor(0.0241), tensor(-0.0266)]",tensor(1.),tensor(False),tensor(1.9900)
1,"[tensor(0.0111), tensor(0.0379), tensor(-0.0292), tensor(0.0257)]",tensor(0.),tensor(False),tensor(1.9900)
2,"[tensor(0.0428), tensor(-0.0466), tensor(-0.0234), tensor(-0.0086)]",tensor(0.),tensor(False),tensor(1.9900)
3,"[tensor(0.0465), tensor(0.1804), tensor(0.0235), tensor(-0.3116)]",tensor(1.),tensor(False),tensor(1.9900)
4,"[tensor(0.0119), tensor(-0.1568), tensor(-0.0286), tensor(0.3091)]",tensor(1.),tensor(False),tensor(1.9900)
5,"[tensor(0.0419), tensor(-0.2414), tensor(-0.0235), tensor(0.2767)]",tensor(1.),tensor(False),tensor(1.9900)
6,"[tensor(0.0501), tensor(0.3751), tensor(0.0173), tensor(-0.5968)]",tensor(0.),tensor(False),tensor(1.9900)
7,"[tensor(0.0088), tensor(0.0388), tensor(-0.0225), tensor(0.0075)]",tensor(1.),tensor(False),tensor(1.9900)
8,"[tensor(0.0371), tensor(-0.0459), tensor(-0.0180), tensor(-0.0234)]",tensor(0.),tensor(False),tensor(1.9900)
9,"[tensor(0.0576), tensor(0.1798), tensor(0.0054), tensor(-0.2987)]",tensor(0.),tensor(False),tensor(1.9900)


## Multi Processing

In [ ]:
from torchdata.dataloader2 import DataLoader2

In [ ]:
%%writefile ../external_run_scripts/spawn_multiproc.py
import torch
import torchdata.datapipes as dp
from torchdata.dataloader2 import DataLoader2,MultiProcessingReadingService
       
class PointlessLoop(dp.iter.IterDataPipe):
    def __init__(self,datapipe=None):
        self.datapipe = datapipe
    
    def __iter__(self):
        while True:
            yield torch.LongTensor(4).detach().clone()
            

if __name__=='__main__':
    from torch.multiprocessing import Pool, Process, set_start_method
    try:
         set_start_method('spawn')
    except RuntimeError:
        pass


    pipe = PointlessLoop()
    pipe = pipe.header(limit=10)
    dls = [DataLoader2(pipe,
            reading_service=MultiProcessingReadingService(
                num_workers = 2
            ))]
    # Setup the Learner
    print('type: ',type(dls[0]))
    for o in dls[0]:
        print(o)

Overwriting ../external_run_scripts/spawn_multiproc.py


In [ ]:
%%python ../external_run_scripts/spawn_multiproc.py
pass

type:  <class 'torchdata.dataloader2.dataloader2.DataLoader2'>
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])


In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()

/usr/local/lib/python3.8/dist-packages/nbdev/export.py:54: UserWarning: Notebook '/home/fastrl_user/fastrl/nbs/07_Agents/02_Continuous/12u_agents.ppo.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
